In [4]:
import pandas as pd
import numpy as np
import os
pd.options.mode.chained_assignment = None 
sp_dir = '/Users/rwang/RMI/Climate Action Engine - Documents/OCI Phase 2'
opem_dir = '/Users/rwang/Documents/OCI+/Downstream/opem'

print('Merging upstream and midstream results...')

# import sqlite3
# connection = sqlite3.connect("../OCI_Database.db")

upstream = pd.read_csv(sp_dir + '/Upstream/upstream_data_pipeline_sp/Postprocessed_Outputs_2/upstream_postprocessed.csv')
midstream = pd.read_csv(sp_dir + '/Upstream/upstream_data_pipeline_sp/Postprocessed_Outputs_2/midstream_postprocessed.csv')


# Calculate Crude to Refinery in bbl/d from Energy Summary tab of OPGEE model. Formula is based on cell G6 in OPEM input tab  
#https://rockmtnins.sharepoint.com/:x:/r/sites/CAE/_layouts/15/Doc.aspx?sourcedoc=%7B5E0994C9-8E35-440B-8BB4-31DF5167F60C%7D&file=OCI%20site%20input%20table%20sources.xlsx&action=default&mobileredirect=true&cid=544ad233-565f-438c-9ce2-d7d5b607b1da

# ('Energy Summary'!E89*1e3/'Energy Summary'!M133)/(158.9873*141.5/(131.5+Results!G34))
upstream['crude_to_refinery(bbl/d)']= (upstream['ES_Crude_output(mmbut/d)']*1e3/upstream['ES_Energy_Density_crude(mmbtu/t)'])/(158.9873*141.5/(131.5+upstream['API gravity']))

# For oil sands, the crude to refinery volume is bigger than the input oil production volume. 
# It's interesting and worth exploring why. Is there a mistake in Raghav's mmbtu/d to boe/d calculation? 
#upstream[['Field_name','year']][upstream['crude_to_refinery(bbl/d)']>upstream['Oil production volume']]

# Calculate NGL_C2 export from the field, formula based on cell G8 in OPEM input tab
# (Flowsheet!W17+Flowsheet!CP17)*1000/(5.61458350903291*20.98*2.2)

upstream['NGL_C2(boed)'] = (upstream['FS_LPG_export_C2(t/d)']+upstream['FS_Ethane_to_Petchem(t/d)'])*1000/(5.61458350903291*20.98*2.2)

# Calculate NGL_C3 export from the field, formula based on cell G9 in OPEM input tab
# Flowsheet!W18*1000000/(42*1920)+Flowsheet!W9*(1000000)*0.75/(1923*42) assuming 75% of LPG exported is C3
upstream['NGL_C3(boed)'] = upstream['FS_LPG_export_C3(t/d)']*1000000/(42*1920)+upstream['FS_LPG_export_LPG(t/d)']*(1000000)*0.75/(1923*42)

# Calculate NGL_C4 export from the field, formula based on cell G10 in OPEM input tab
# Flowsheet!W19*1000000/(42*2213)+Flowsheet!W9*(1000000)*0.25/(1923*42) assuming 25% of LPG exported is C4
upstream['NGL_C4(boed)'] = upstream['FS_LPG_export_C4(t/d)']*1000000/(42*2213)+upstream['FS_LPG_export_LPG(t/d)']*(1000000)*0.25/(1923*42)

# Calculate NGL_C5+ export from the field, formula based on cell G11 in OPEM input tab
# Assuming all NGL minus C2, C3 and C4 is C5+
# ('Energy Summary'!E87-(((Flowsheet!W17+Flowsheet!CP17)*'Energy Summary'!M141) + 
# (Flowsheet!W18*'Energy Summary'!M142) + (Flowsheet!W19*'Energy Summary'!M143) + 
#(Flowsheet!W9*84950/1923)))/(42*.110)
upstream['NGL_C5(boed)'] = (upstream['ES_NGL_output(mmbtu/d)']-(((upstream['FS_LPG_export_C2(t/d)']+ \
                        upstream['FS_Ethane_to_Petchem(t/d)'])*upstream['ES_Energy_Density_C2(mmbtu/t)']) + \
                        (upstream['FS_LPG_export_C3(t/d)']*upstream['ES_Energy_Density_C3(mmbtu/t)']) + \
                        (upstream['FS_LPG_export_C4(t/d)']*upstream['ES_Energy_Density_C4(mmbtu/t)']) + \
                        (upstream['FS_LPG_export_LPG(t/d)']*84950/1923)))/(42*.110)

# Calculate petcoke from the field, formula based on cell G12 in OPEM input tab
# 1000*(Flowsheet!HG7-'Energy Summary'!E77/'Energy Summary'!M135)
upstream['petcoke(kg/d)'] = 1000*(upstream['FS_Petcoke_to_stock(t/d)']-upstream['ES_Petcoke_fuel(mmbtu/d)']/upstream['ES_Energy_Density_petcoke(mmbtu/t)'])




Merging upstream and midstream results...
